In [1]:
# Import libraries

import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import linear_model
import logging
import pandas as pd
import numpy as np
from numpy import random
import itertools

import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier, Perceptron, PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
import time
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt
import nltk
nltk.download('wordnet')
from sklearn.model_selection import RandomizedSearchCV

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import roc_auc_score

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jasoncharnock/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# Prediction model using CountVectorizer() and LogisticRegression(), which is insufficient to train all data from training set 

# Opening JSON files

f = open('train.json',)
data = json.load(f)

g = open('test.json',)
test = json.load(g)

# Pre-processing training/validation data 
    # It combines the columns that are given in the original dataset, to have only one independent variable column
data = pd.DataFrame(data)
data['year'] = data['year'].astype(str)
data['labels']=data[['title', 'abstract', 'year', 'venue']].apply(lambda x:','.join(x), axis = 1) 
merged_data=data.drop(labels =['title', 'abstract', 'year', 'venue', 'paperId'], axis = 1)
merged_data.head()

# This replaces capital letters, and some symbols. It lowers all the texts, strips and splits and converts it to strings
merged_data['labels'] = merged_data['labels'].str.replace('[^A-Za-z]',' ')
merged_data['labels'] = merged_data['labels'].str.lower()
merged_data["labels"] = merged_data["labels"].str.strip()
merged_data["labels"] = merged_data["labels"].str.split() 
merged_data['labels'] = [','.join(map(str, l)) for l in merged_data['labels']]


# Pre-processing test data (same process as above, but for test dataset)

test = pd.DataFrame(test)
test['year'] = test['year'].astype(str)
test['labels']=test[['title', 'abstract', 'year', 'venue']].apply(lambda x:','.join(x), axis = 1) 
merged_test=test.drop(labels =['title', 'abstract', 'year', 'venue'], axis = 1)
merged_test.head()

merged_test['labels'] = merged_test['labels'].str.replace('[^A-Za-z]',' ')
merged_test['labels'] = merged_test['labels'].str.lower()
merged_test["labels"] = merged_test["labels"].str.strip()
merged_test["labels"] = merged_test["labels"].str.split() 
merged_test['labels'] = [','.join(map(str, l)) for l in merged_test['labels']]

# Replace stopwords, symbols
    # cleans out all the stopwords (the, a, an, etc.)
    # removes all symbols and numbers

replace = re.compile('[/(){}\[\]\|@,;]')
replace_symbols = re.compile('[^0-9a-z #+_]')
replace_stopwords = set(stopwords.words('english'))

def clean_labels(labels):
    
    labels = labels.lower() # lowercase labels
    labels = replace.sub(' ', labels) # replace REPLACE_BY_SPACE_RE symbols by space in labels
    labels = replace_symbols.sub('', labels) # delete symbols which are in BAD_SYMBOLS_RE from labels
    labels = ' '.join(word for word in labels.split() if word not in replace_stopwords) # delete stopwors from labels
    return labels
    
merged_data['labels'] = merged_data['labels'].apply(clean_labels) # applies the above loop to the data
    
merged_test['labels'] = merged_test['labels'].apply(clean_labels) # applies the loop to test data

w_tokenizer = nltk.tokenize.WhitespaceTokenizer() # seperates the sentences to get single words
lemmatizer = nltk.stem.WordNetLemmatizer() # lemmatizes text
def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

merged_data['labels'] = merged_data['labels'].apply(lemmatize_text)

merged_data['labels'] = [','.join(map(str, l)) for l in merged_data['labels']]


/var/folders/c4/vwzjd2dn1ndg85vgr9jy1xv80000gn/T/ipykernel_23772/2075360187.py:18: FutureWarning: The default value of regex will change from True to False in a future version.
  merged_data['labels'] = merged_data['labels'].str.replace('[^A-Za-z]',' ')
/var/folders/c4/vwzjd2dn1ndg85vgr9jy1xv80000gn/T/ipykernel_23772/2075360187.py:33: FutureWarning: The default value of regex will change from True to False in a future version.
  merged_test['labels'] = merged_test['labels'].str.replace('[^A-Za-z]',' ')


In [ ]:
# selects only the authors that are appearing in the dataset at least 4 times
merged_data = merged_data.groupby('authorId').filter(lambda x : x['authorId'].shape[0]>=4)




In [4]:
merged_data['labels'] # preview of the data

1        bigram,bilstms,two,neural,network,sequential,m...
6        algorithm,simultaneously,bracketing,parallel,t...
8        limsi,submission,wmt,qe,task,paper,describes,l...
10       graph,auto,encoder,model,derivational,morpholo...
13       transfer,learning,impact,linguistic,knowledge,...
                               ...                        
12104    gender,bias,multilingual,embeddings,cross,ling...
12112    improving,semantic,composition,offset,inferenc...
12116    two,stage,approach,extending,event,detection,n...
12119    parameter,born,equal,attention,mostly,need,tra...
12120    reconstruction,word,embeddings,sub,word,parame...
Name: labels, Length: 3268, dtype: object

In [26]:
# Determine independent and dependent variables 

X = merged_data['labels']
y = merged_data['authorId']

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.15, random_state=42)
vectorizer = HashingVectorizer(ngram_range=(1,2), n_features=2**18) 
# applies the hashing vectorizer (found to be the best for large text datasets)
X_train_hashed = vectorizer.transform(X_train)
tfidf_transformer=TfidfTransformer() # transforms the hashed vectorized texted by using tfidf transformer
# TFIDF works by proportionally increasing the number of times a word appears in the document 
# but is counterbalanced by the number of documents in which it is present
# https://www.analyticsvidhya.com/blog/2021/07/bag-of-words-vs-tfidf-vectorization-a-hands-on-tutorial/
X_train = tfidf_transformer.fit_transform(X_train_hashed)
X_test_hashed = vectorizer.transform(X_test)
X_test = tfidf_transformer.transform(X_test_hashed)
X_train

<2777x262144 sparse matrix of type '<class 'numpy.float64'>'
	with 401794 stored elements in Compressed Sparse Row format>

In [20]:
# This trains the model

epoch = 5 # the amount of times that it will run
batchsize = 500 # the amount of data you put in per batch
model = SGDClassifier() # The classifier to use, this is supposed to be best applicable to large datasets
batches = int(X_train.shape[0]/batchsize) + 1
samples = X_train.shape[0]
for i in range(epoch):
    for j in range(batches):
        print('in j...', j, j*batchsize, '----2is:',samples, (j+1)*batchsize )
        model.partial_fit(X_train[j*batchsize:min(samples,(j+1)*batchsize)], 
                          Y_train[j*batchsize:min(samples,(j+1)*batchsize)], 
                          classes=np.unique(y))
print ("Accuracy on testing data :", model.score(X_test, Y_test))

in j... 0 0 ----2is: 2777 500
in j... 1 500 ----2is: 2777 1000
in j... 2 1000 ----2is: 2777 1500
in j... 3 1500 ----2is: 2777 2000
in j... 4 2000 ----2is: 2777 2500
in j... 5 2500 ----2is: 2777 3000
in j... 0 0 ----2is: 2777 500
in j... 1 500 ----2is: 2777 1000
in j... 2 1000 ----2is: 2777 1500
in j... 3 1500 ----2is: 2777 2000
in j... 4 2000 ----2is: 2777 2500
in j... 5 2500 ----2is: 2777 3000
in j... 0 0 ----2is: 2777 500
in j... 1 500 ----2is: 2777 1000
in j... 2 1000 ----2is: 2777 1500
in j... 3 1500 ----2is: 2777 2000
in j... 4 2000 ----2is: 2777 2500
in j... 5 2500 ----2is: 2777 3000
in j... 0 0 ----2is: 2777 500
in j... 1 500 ----2is: 2777 1000
in j... 2 1000 ----2is: 2777 1500
in j... 3 1500 ----2is: 2777 2000
in j... 4 2000 ----2is: 2777 2500
in j... 5 2500 ----2is: 2777 3000
in j... 0 0 ----2is: 2777 500
in j... 1 500 ----2is: 2777 1000
in j... 2 1000 ----2is: 2777 1500
in j... 3 1500 ----2is: 2777 2000
in j... 4 2000 ----2is: 2777 2500
in j... 5 2500 ----2is: 2777 3000
Accur

In [7]:
# https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction
# https://towardsdatascience.com/multi-class-text-classification-model-comparison-and-selection-5eb066197568
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html

# https://scikit-learn.org/stable/auto_examples/applications/plot_out_of_core_classification.html#sphx-glr-auto-examples-applications-plot-out-of-core-classification-py
# https://medium.com/mlearning-ai/out-of-core-multi-label-text-classification-with-scikit-learn-14afa4c1bb75
# https://towardsdatascience.com/how-to-make-sgd-classifier-perform-as-well-as-logistic-regression-using-parfit-cc10bca2d3c4

In [21]:
new_data = merged_test['labels'] # the test data labels that we will apply to transform to numerical
X_new = vectorizer.transform(new_data) # transforms the data by using the vectorizer
y_pred = model.predict(X_new) # predicts the new values of the test set

In [24]:
merged_test['prediction'] = y_pred.tolist() # puts the predicted data to list
final = merged_test.set_axis(['paperId', 'labels', 'authorId'], axis=1, inplace=False) # changes the axis labels
final = final.drop(labels = ['labels'], axis = 1) # drops the labels column to get final result of only paperId & authorId

In [25]:
final

,paperId,authorId
0,86e1aaa0c47659e08a896e9889384eb1e5401e6a,143655216
1,8d3076c38f56df22052567f4783c670d8e860f09,14488816
2,7c400ee676d427eeda1aad5c1c54c316f0b9773d,1788050
3,185e7d2a761594451b02ace240356dadad2aef78,3115181
4,e4363d077a890c8d5c5e66b82fe69a1bbbdd5c80,1989344
...,...,...
6526,069ebed0ba7adec30faa5c5e008053cf3eefc589,1678591
6527,b6e9fdc3e7bc4d379ee733b07199fe2a8336dd94,47210642
6528,5019da491732e412fafea4e1511818fd684cc1f1,1750769
6529,eca16c1c776406abd0d966653a705f945bd4b520,1779225
